In [193]:
import pandas as pd

# Load data
tesla = pd.read_csv("TSLA_5_Minutes.csv")

# Combine 'Date' and 'Time' into a datetime column
tesla['Datetime'] = pd.to_datetime(tesla['Date'] + ' ' + tesla['Time'], dayfirst=True)

# Filter to desired date range (inclusive)
start_date = pd.to_datetime('2017-09-06')
end_date = pd.to_datetime('2018-09-06')

tesla = tesla[(tesla['Datetime'] >= start_date) & (tesla['Datetime'] <= end_date)]

# Optional: set datetime as index for time-series ops
tesla = tesla.set_index('Datetime')

print(tesla)

                       Open    High     Low   Close  Volume        Date  \
Datetime                                                                  
2017-09-06 09:35:00  349.65  350.98  347.28  347.28  322829  06-09-2017   
2017-09-06 09:40:00  347.49  348.39  346.50  346.88  111292  06-09-2017   
2017-09-06 09:45:00  347.00  347.78  346.00  346.32   72377  06-09-2017   
2017-09-06 09:50:00  346.35  346.75  344.50  345.34  114883  06-09-2017   
2017-09-06 09:55:00  345.49  346.49  345.24  345.77   67583  06-09-2017   
...                     ...     ...     ...     ...     ...         ...   
2018-09-05 15:40:00  281.89  282.18  281.35  281.55   37374  05-09-2018   
2018-09-05 15:45:00  281.36  281.56  280.22  280.48   80286  05-09-2018   
2018-09-05 15:50:00  280.46  280.65  280.25  280.33   84565  05-09-2018   
2018-09-05 15:55:00  280.32  280.56  280.02  280.35  119435  05-09-2018   
2018-09-05 16:00:00  280.39  280.71  280.26  280.70  163188  05-09-2018   

                        

*******************************************************************

### Breakout strategy with time avoidance, trade freq, dynamic trheshold. 

In [201]:
import numpy as np
import matplotlib.pyplot as plt

# --- Parameters ---
stop_loss_pct = 0.027        # 2% hard stop
take_profit_pct = 0.05      # 4% trailing stop
tesla["Position"] = "None"
tesla["PnL"] = 0.0

# --- Step 1: Compute 20-day average volume ---
tesla['Day'] = tesla.index.date
daily_vol = tesla.groupby('Day')['Volume'].sum().rolling(20).mean()
tesla['20D_Avg_Volume'] = tesla['Day'].map(daily_vol)

# --- Step 2: Group by trading day ---
grouped = tesla.groupby(tesla.index.date)
trades = []

for date, group in grouped:
    group = group.sort_index()
    if len(group) < 20 or group['20D_Avg_Volume'].isna().all():
        continue

    # --- Opening Range: 9:30 - 10:00 AM ---
    opening_range = group.between_time("09:30", "10:00")
    after_opening = group.between_time("10:01", "16:00")

    if opening_range.empty or after_opening.empty:
        continue

    high_or = opening_range['High'].max()
    low_or = opening_range['Low'].min()

    # --- Dynamic breakout threshold ---
    or_range = high_or - low_or
    or_pct = or_range / low_or

    if or_pct < 0.01:  # if OR is < 1%, consider this a tight range
        threshold = 0.02  # require a 1.5% breakout
    else:
        threshold = 0.01   # require a 1.0% breakout

    long_break_price = high_or * (1 + threshold)
    short_break_price = low_or * (1 - threshold)

    long_trade_done = False
    short_trade_done = False

    for i in range(len(after_opening)):
        row = after_opening.iloc[i]
        idx = after_opening.index[i]

        avg_daily_volume = row['20D_Avg_Volume']
        if pd.isna(avg_daily_volume):
            continue
        vol_threshold = (1.5 * avg_daily_volume / 78)  # 78 5-min bars/day

        # --- LONG TRADE ---
        if not long_trade_done and row['High'] >= long_break_price and row['Volume'] > vol_threshold:
            entry_price = row['Close']
            stop_loss = entry_price * (1 - stop_loss_pct)
            max_price = entry_price
            trade_type = 'Long'
            tesla.at[idx, 'Position'] = trade_type

            for j in range(i + 1, len(after_opening)):
                forward = after_opening.iloc[j]
                max_price = max(max_price, forward['High'])

                if forward['Low'] <= stop_loss:
                    exit_price = stop_loss
                    pnl = exit_price - entry_price
                    trailing_stop = None
                    break

                trailing_stop = max_price * (1 - take_profit_pct)
                if forward['Low'] <= trailing_stop:
                    exit_price = trailing_stop
                    pnl = exit_price - entry_price
                    break
            else:
                exit_price = after_opening.iloc[-1]['Close']
                pnl = exit_price - entry_price
                trailing_stop = None

            tesla.at[idx, 'PnL'] = pnl
            trades.append({
                'Date': idx.date(),
                'Entry': entry_price,
                'Exit': exit_price,
                'PnL': pnl,
                'Position': trade_type,
                'Stop_Loss': stop_loss,
                'Max_High': max_price,
                'Min_Low': None,
                'Trailing_Stop': trailing_stop,
                'Breakout_Ref': high_or,
                'Volume_At_Entry': row['Volume']
            })
            long_trade_done = True

        # --- SHORT TRADE ---
        elif not short_trade_done and row['Low'] <= short_break_price and row['Volume'] > vol_threshold:
            entry_price = row['Close']
            stop_loss = entry_price * (1 + stop_loss_pct)
            min_price = entry_price
            trade_type = 'Short'
            tesla.at[idx, 'Position'] = trade_type

            for j in range(i + 1, len(after_opening)):
                forward = after_opening.iloc[j]
                min_price = min(min_price, forward['Low'])

                if forward['High'] >= stop_loss:
                    exit_price = stop_loss
                    pnl = entry_price - exit_price
                    trailing_stop = None
                    break

                trailing_stop = min_price * (1 + take_profit_pct)
                if forward['High'] >= trailing_stop:
                    exit_price = trailing_stop
                    pnl = entry_price - exit_price
                    break
            else:
                exit_price = after_opening.iloc[-1]['Close']
                pnl = entry_price - exit_price
                trailing_stop = None

            tesla.at[idx, 'PnL'] = pnl
            trades.append({
                'Date': idx.date(),
                'Entry': entry_price,
                'Exit': exit_price,
                'PnL': pnl,
                'Position': trade_type,
                'Stop_Loss': stop_loss,
                'Max_High': None,
                'Min_Low': min_price,
                'Trailing_Stop': trailing_stop,
                'Breakout_Ref': low_or,
                'Volume_At_Entry': row['Volume']
            })
            short_trade_done = True

        # Stop after both trades executed
        if long_trade_done and short_trade_done:
            break

# --- Metrics on full dataset ---
tesla['ReturnPctTrade'] = tesla['PnL'] / tesla['Open'] * 100
tesla['Cumulative_PnL'] = tesla['PnL'].cumsum()


In [202]:
# --- Convert trades to DataFrame ---
trade_df = pd.DataFrame(trades)

if trade_df.empty:
    print("No trades executed.")
else:
    trade_df['ReturnPctTrade'] = (trade_df['PnL'] / trade_df['Entry']) * 100
    trade_df['Cumulative_PnL'] = trade_df['PnL'].cumsum()

    # Summary metrics
    total_profit = trade_df['PnL'].sum()
    avg_return = trade_df['ReturnPctTrade'].mean()
    num_winners = (trade_df['PnL'] > 0).sum()
    num_losers = (trade_df['PnL'] < 0).sum()
    win_rate = (num_winners / len(trade_df)) * 100
    avg_win = trade_df[trade_df['PnL'] > 0]['ReturnPctTrade'].mean()
    avg_loss = trade_df[trade_df['PnL'] < 0]['ReturnPctTrade'].mean()
    profit_factor = trade_df[trade_df['PnL'] > 0]['PnL'].sum() / abs(trade_df[trade_df['PnL'] < 0]['PnL'].sum()) if num_losers > 0 else float('inf')

    print(f"\n🔍 Breakout Strategy Performance Summary")
    print(f"-----------------------------------------")
    print(f"Total Profit:          ${total_profit:.2f}")
    print(f"Total Trades:          {len(trade_df)}")
    print(f"Winning Trades:        {num_winners}")
    print(f"Losing Trades:         {num_losers}")
    print(f"Win Rate:              {win_rate:.2f}%")
    print(f"Avg % Return / Trade:  {avg_return:.2f}%")
    print(f"Avg Win %:             {avg_win:.2f}%")
    print(f"Avg Loss %:            {avg_loss:.2f}%")
    print(f"Profit Factor:         {profit_factor:.2f}")



🔍 Breakout Strategy Performance Summary
-----------------------------------------
Total Profit:          $84.69
Total Trades:          108
Winning Trades:        59
Losing Trades:         43
Win Rate:              54.63%
Avg % Return / Trade:  0.24%
Avg Win %:             1.43%
Avg Loss %:            -1.37%
Profit Factor:         1.46
